In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'inside_diameter_metric'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [7]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['InsideDiameter']
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

4468


In [9]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [14]:
start=r'(?i)(^.{0,7})|()'
df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(\d+\s?mm)|()''' 
df['name'] = df['start'].apply(lambda x: re_extract(pattern, str(x)))
# df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d+\s?mm)|()'
df['matches']=df['InsideDiameter'].apply(lambda x: re_extract(pat, str(x)))
custom=df[df['matches'].astype(str)!='[]']
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)(?<=\d)mm',' mm',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]

179


In [23]:
two=df[(df['matches'].astype(str)=='[]')]
pattern=r'''(?i)(\d+\s?mm)|()''' 
two['name'] = two['product_name'].apply(lambda x: re_extract(pattern, str(x)))
print(len(two[two['name'].astype(str)=='[]']))
two[two['name'].astype(str)=='[]']

3867


buckets external_id  \
0     Compressors Accessories & Replacement Parts       40511   
1     Compressors Accessories & Replacement Parts       40490   
2     Compressors Accessories & Replacement Parts       40492   
3     Compressors Accessories & Replacement Parts       40491   
4     Compressors Accessories & Replacement Parts       40493   
...                                           ...         ...   
4463  Compressors Accessories & Replacement Parts    405784RE   
4464  Compressors Accessories & Replacement Parts    405786RE   
4465  Compressors Accessories & Replacement Parts    405787RE   
4466  Compressors Accessories & Replacement Parts     40578RE   
4467  Compressors Accessories & Replacement Parts     509005U   

                                                                         product_name  \
0                                                        1/16 Viton O-Ring Assortment   
1                                              1/16 - 1/8 Buna-N 90 O-Ring Assortment   
2                                                    3/32 Buna-N 90 O-Ring Assortment   
3                                                    1/16 Buna-N 90 O-Ring Assortment   
4                                                     1/8 Buna-N 90 O-Ring Assortment   
...                                                                               ...   
4463       Refill Kit for 1/4 - 5/8 Kim-Krimp 'Z' Series Face Seal Fitting Assortment   
4464                 Refill Kit for 1/4 - 1/2 Kim-Krimp 'Z' Series Fitting Assortment   
4465          Refill Kit for 1/4 - 1/2 Kim-Krimp 'Z' Series O-Ring Fitting Assortment   
4466  Refill Kit for Straight Thread O-Ring Kim-Krimp Hose End Assortment (1/4 - 1/2)   
4467                             Buna-N 90 O-Ring Assortment with O-Ring Gauge Bundle   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            75 Durometer. Manufactured from fluorocarbon rubber. Temperature range: -20° to 400°F. Has maximum resistance to deterioration by heat and fluids. Recognized by brown color.   
1                                                                                                                                                                                   

In [24]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [25]:
na=two[two['name'].astype(str)=='[]']
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na)+len(matchcustom))

4046


In [26]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))


# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [27]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [28]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 